In [92]:
import numpy as np
from scipy import stats

import plotly.offline as py
import plotly.graph_objs as go

In [118]:
# Plot layout
layout = go.Layout(
    scene = dict(
        camera = dict(
            up=dict(x=0, y=0, z=1),
            center=dict(x=0, y=0, z=0),
            eye=dict(x=1.8, y=0, z=-0.45)
        ),
        xaxis = dict(
            title='',
            showgrid=False,
            zeroline=False,
            mirror=False,
            showline=False,
            ticks='',
            showticklabels=False
        ),
        yaxis = dict(
            title='',
            showgrid=False,
            zeroline=False,
            mirror=False,
            showline=False,
            ticks='',
            showticklabels=False
        ),
        zaxis = dict(
            title='',
            showgrid=False,
            zeroline=False,
            showline=False,
            ticks='',
            showticklabels=False
        )
    )
)

In [119]:
n = 300

x = np.linspace(-1.8, 1.8, n)
y = np.linspace(-1.8, 1.8, n)

X, Y = np.meshgrid(x, y)

XY = np.empty((n * n, 2))
XY[:, 0] = X.flatten()
XY[:, 1] = Y.flatten()

#------------------------------------------------------------------------

# Z_native
cov = np.array([[0.055, 0.002],
               [0.005, 0.055]])
dist = stats.multivariate_normal(np.array([0.5, -1.1]), cov)
Z_native = dist.pdf(XY).reshape((n, n)) * 0.5

# cov = np.array([[0.05, 0.002],
#                [0.003, 0.05]])
# dist = stats.multivariate_normal(np.array([-0.5, -0.8]), cov)
# Z_native += dist.pdf(XY).reshape((n, n)) * 0.15

# cov = np.eye(2) * 0.08
# dist = stats.multivariate_normal(np.array([-0.6, -0.6]), cov)
# Z_native += dist.pdf(XY).reshape((n, n)) * 0.15

#-------------------------------------------------------------------------

# Z_new
cov = np.eye(2) * 0.06
dist = stats.multivariate_normal(np.array([-0.98, 0.98]), cov)
Z_new = dist.pdf(XY).reshape((n, n)) * 0.6

cov = np.eye(2) * 0.12
dist = stats.multivariate_normal(np.array([-0.3, 0.6]), cov)
Z_new += dist.pdf(XY).reshape((n, n)) * 0.6

cov = np.eye(2) * 0.12
dist = stats.multivariate_normal(np.array([0.5, 0.2]), cov)
Z_new += dist.pdf(XY).reshape((n, n)) * 0.3

cov = np.eye(2) * 0.1
dist = stats.multivariate_normal(np.array([0.9, -0.6]), cov)
Z_new += dist.pdf(XY).reshape((n, n)) * 0.15

In [120]:
promisc = np.empty_like(Z_native)

for i in range(len(Z_native[0, :])):
    for j in range(len(Z_native[:, 0])):
        if (Z_native[i, j] >= 0.005) and (Z_new[i, j] >= 0.005):
        
            if Z_native[i, j] >= Z_new[i, j]:
                promisc[i, j] = Z_new[i, j]*0.99
            else:
                promisc[i, j] = Z_native[i, j]#*0.99
        else:
            promisc[i, j] = None

promisc

array([[nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       ...,
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan]])

In [129]:
# Blue used in native protein: #3273BB
blues = [[0, '#222277'], [1.0, '#3273CB']]
# Red used in evolved protein: #A01F1D
reds = [[0, '#772222'], [1.0, '#C01F1D']]
# promisc
c_promisc = [[0, '#111100'], [0.2, '#FFFF00'], [1.0, '#FFFF00']]


# Lighting
lighting = dict(ambient=0.5, diffuse=1, specular=0.2, roughness=0.5)

In [134]:
data = [
    go.Surface(z=Z_native, opacity=0, colorscale=blues, lighting=lighting),
    go.Surface(z=Z_new, opacity=1, colorscale=reds, lighting=lighting)
]

fig = go.Figure(data=data, layout=layout)

py.plot(fig, filename='final_landscape.html')

'file:///Users/Patrick/git/fitness_landscape/final_landscape.html'

In [132]:
# Bad rendering with transparency

# data = [
#     go.Surface(z=Z_native, opacity=0.92, colorscale=blues, lighting=lighting),
#     go.Surface(z=Z_new, opacity=0.92, colorscale=reds, lighting=lighting),
#     go.Surface(z=promisc, opacity=1, colorscale=c_promisc, lighting=dict(ambient=1, specular=0))
# ]

# fig = go.Figure(data=data, layout=layout)

# py.plot(fig, filename='final_landscape_with_rop.html')

'file:///Users/Patrick/git/fitness_landscape/final_landscape_with_rop.html'

In [131]:
data = [
    go.Surface(z=Z_native, opacity=0, colorscale=blues, lighting=lighting),
    go.Surface(z=Z_new, opacity=0, colorscale=reds, lighting=lighting),
    go.Surface(z=promisc, opacity=0.6, colorscale=c_promisc, lighting=dict(ambient=1, specular=0))
]

fig = go.Figure(data=data, layout=layout)

py.plot(fig, filename='region_of_promisc_overlay.html')

'file:///Users/Patrick/git/fitness_landscape/region_of_promisc_overlay.html'

In [ ]:
# Screenshot: 420, 720; 550, 400